### Use when in Colab

In [1]:
# !git clone https://github.com/alealcoforado/zeroberto.git
# %cd zeroberto

In [2]:
google_colab = False

# Dependencies

In [3]:
%load_ext autoreload
%autoreload 2
import datasets_handler
import evaluation_metrics
from zeroberto import ZeroBERTo
import gc
import pandas as pd
from datasets import load_dataset, Dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

/Users/alealcoforado/Documents/Projetos/ZeroBERTo/zeroberto/datasets_handler.py:246: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  def splitDocuments(docs: pd.Series()) -> list():
0it [00:00, ?it/s]


In [4]:
if google_colab:
    from google.colab import drive
    drive.mount('/content/drive')
    # dataset_path = '/content/drive/Othercomputers/My MacBook Pro/Datasets/folhauol/folhauol_clean_df_articles.csv'
    # dataset_path = '/content/drive/MyDrive/Maquina de London/clean_df_articles_folhauol.csv'
    zeroshot_data_path = './'
else:
    dataset_path = None
    zeroshot_data_path = None

# Data

In [5]:
which_dataset = 'ag_news' 
max_inferences = 100
random_state = 333
le = LabelEncoder()

raw_data, data_col, class_col = datasets_handler.getDataset(which_dataset,path=dataset_path,test_only=True)
classes_list = list(raw_data[class_col].unique())
le.fit(raw_data[class_col])
raw_data = evaluation_metrics.Encoder(raw_data,labelEncoder=le,columnsToEncode=[class_col])
data_to_label = raw_data.sample(max_inferences,random_state=random_state).sort_index()
print(classes_list,data_to_label.shape)

Found cached dataset ag_news (/Users/alealcoforado/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)


  0%|          | 0/2 [00:00<?, ?it/s]

['business', 'science and technology', 'sports', 'world'] (100, 4)


# Parameters

In [6]:
zeroshot_method = "dotproduct"  

zeroshot_config = {
    'similarity_model' : 'sentence-transformers/all-roberta-large-v1', #'sentence-transformers/stsb-xlm-r-multilingual', # model for semantic similarity
    'setfit_model' :  'sentence-transformers/all-roberta-large-v1',#'sentence-transformers/stsb-xlm-r-multilingual',# 'ricardo-filho/bert-base-portuguese-cased-nli-assin-2' # model for contrastive learning and classification
    'split':"zeroshot", ### fewshot (uses real training data) or zeroshot (uses previously selected training data)
    
    'class_col':class_col, ### which column of the dataset has the classes
    'data_col':data_col, ### which column of the dataset has the data
    'dataset':which_dataset, ### which dataset we are using
    'labeling_method':zeroshot_method, ### how the first labeling should be executed
    'max_inferences':max_inferences, ### sample size from full dataset (number of records)
    'classes':classes_list, ### classes from the dataset
    'template': "this text is mainly about {}.", ### prompt that will be used for semantic search
    'random_state':random_state, ### random state for reproducibility
    'top_n': 2, ### how many best predictions from zeroshot should be recovered
    'training_examples': 2, ### how many examples per class should be used for training
    "batch_size" : 4, ### contrastive training batch size
    "num_pairs" : 2, ### number of text pairs that will be generated in contrastive learning
    "num_epochs" : 1, ### number of epochs training will be conducted
    # 'keep_body_frozen_setfit': True,     # Unfreeze head and freeze body -> head-only training
    'keep_body_frozen_setfit' : False, # Unfreeze head and body -> end-to-end training
    'label_encoder' : le, ### label encoder to use for encoding categorial data into int
    'average_sentence_embeddings':False, ### whether documents should be split by . and their embeddings be a mean of sentences embeddings
    'self_training_epochs' : 0, ### how many ZeroBERTo training steps have been executed

    'st_train_epochs': 10, ### sentence-transformer TSDAE fine-tuning --- do not use
    'st_train_batch_size': 20, ### sentence-transformer TSDAE fine-tuning --- do not use
}

# Model

In [7]:
model = ZeroBERTo(classes_list=zeroshot_config['classes'],labeling_dataset=data_to_label,
                embeddingModel=zeroshot_config['similarity_model'],config=zeroshot_config)
       

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


queries
3
4


# Labeling

In [8]:
%%time
if (model.config['split']=='zeroshot'): ###  selecting training data
    model.runLabeling()
    model.evaluateLabeling()
    model.getLabelingMetrics()
    model.saveLabelingResults(local_path=zeroshot_data_path)
# model.config['exec_time'] = '2023_04_16__11_48_09' #### comment above and use this if labeling is already done

     prediction_code              prediction
122                1  science and technology
549                0                business
678                3                   world
747                2                  sports
top 1: {'accuracy': 0.75}
top 2: {'accuracy': 0.875}
top 3: {'accuracy': 0.8333333333333334}
top 4: {'accuracy': 0.875}
top 5: {'accuracy': 0.85}
top 6: {'accuracy': 0.8333333333333334}
top 7: {'accuracy': 0.8214285714285714}
top 8: {'accuracy': 0.8125}
top 9: {'accuracy': 0.8333333333333334}
top 10: {'accuracy': 0.825}
top 11: {'accuracy': 0.7954545454545454}
top 12: {'accuracy': 0.7916666666666666}
top 13: {'accuracy': 0.7884615384615384}
top 14: {'accuracy': 0.8}
top 15: {'accuracy': 0.8103448275862069}
top 16: {'accuracy': 0.8032786885245902}
top 100: {'accuracy': 0.75}
predictions_and_probabilities_test_2023_04_17__20_20_25.csv
zeroshot_config_test_2023_04_17__20_20_25.csv
CPU times: user 23 s, sys: 2.03 s, total: 25 s
Wall time: 41.8 s


Constrative Learning Data for First Training

In [9]:
df_train,df_test = datasets_handler.splitDataset(model.labeling_dataset,model.config) 
train_dataset,test_dataset = datasets_handler.buildDatasetDict(df_train,df_test)
print(train_dataset.shape,test_dataset.shape)
df_train
# print(type(df_test)==type(pd.DataFrame()))

(8, 2) (92, 2)


/Users/alealcoforado/Documents/Projetos/ZeroBERTo/zeroberto/datasets_handler.py:246: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  def splitDocuments(docs: pd.Series()) -> list():
/Users/alealcoforado/Documents/Projetos/ZeroBERTo/zeroberto/datasets_handler.py:214: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data[data_col] = test_data[data_col].apply(str)
/Users/alealcoforado/Documents/Projetos/ZeroBERTo/zeroberto/datasets_handler.py:215: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

,text,class_code
Unnamed: 0,,
3325,Computer Associates to buy Netegrity Corporate...,0
2600,Fannie Mae agrees to accounting changes Fannie...,0
7421,HP Shifting Last of Itanium Engineers Hewlett-...,1
6167,US Spacecraft to Probe Origin of Gamma Rays Fa...,1
3520,"Yanks shock Twins, earn date with Sox: Another...",2
747,Argentina Beats U.S. Men's Basketball Team Arg...,2
5287,Ivory Coast Leader Urges End to Violence ABID...,3
4813,Thai PM to Address Nation as More Bombs Hit So...,3


# First Training 

In [12]:
### build trainer for contrastive learning
model.buildTrainer(train_dataset)

### train
model.contrastive_train(train_dataset=train_dataset)
gc.collect()

# model.predict(test_dataset['text'])
y_probs = model.predict_proba(test_dataset['text'])


0it [00:00, ?it/s]
Applying column mapping to training dataset
***** Running training *****
  Num examples = 32
  Num epochs = 1
  Total optimization steps = 8
  Total train batch size = 4


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/8 [00:00<?, ?it/s]

Predicting probabilities of 92 documents


In [13]:
setfit_all_metrics = evaluation_metrics.get_metrics(model.y_pred, test_dataset["class_code"])
print(setfit_all_metrics)
print(classification_report(model.y_pred, test_dataset['class_code'],target_names=classes_list))
setfit_exec_time  = evaluation_metrics.saveResults(model.config,setfit_all_metrics,None)
print(model.config)

{'weighted': [{'accuracy': 0.6847826086956522}, {'precision': 0.695311167945439}, {'recall': 0.6847826086956522}, {'f1': 0.6797277037157097}], 'macro': [{'accuracy': 0.6847826086956522}, {'precision': 0.7006410256410256}, {'recall': 0.6811334498834499}, {'f1': 0.6804251200802924}]}
                        precision    recall  f1-score   support

              business       0.79      0.56      0.66        34
science and technology       0.36      0.53      0.43        15
                sports       0.80      0.94      0.86        17
                 world       0.77      0.77      0.77        26

              accuracy                           0.68        92
             macro avg       0.68      0.70      0.68        92
          weighted avg       0.72      0.68      0.69        92

metrics_setfit_2023_04_17__20_28_56.csv
config_setfit_2023_04_17__20_28_56.csv
{'similarity_model': 'sentence-transformers/all-roberta-large-v1', 'setfit_model': 'sentence-transformers/all-roberta-large

# Next Training

In [16]:
new_train_data = model.selectTrainingData(evaluate=True, y_test=test_dataset['class_code'],method='top_n',top_n=4)
df_train = pd.DataFrame(test_dataset).loc[new_train_data.index.to_list()]
train_dataset, _ = datasets_handler.buildDatasetDict(df_train)
train_dataset


top 1: {'accuracy': 1.0}
top 2: {'accuracy': 0.875}
top 3: {'accuracy': 0.9166666666666666}
top 4: {'accuracy': 0.9375}
top 5: {'accuracy': 0.9}
top 6: {'accuracy': 0.8333333333333334}
top 7: {'accuracy': 0.7857142857142857}
top 8: {'accuracy': 0.8125}
top 9: {'accuracy': 0.8055555555555556}
top 10: {'accuracy': 0.775}
top 11: {'accuracy': 0.7727272727272727}
top 12: {'accuracy': 0.7708333333333334}
top 13: {'accuracy': 0.7692307692307693}
top 14: {'accuracy': 0.7678571428571429}
top 15: {'accuracy': 0.75}
top 16: {'accuracy': 0.7619047619047619}
top 92: {'accuracy': 0.6847826086956522}
Data selected for training: 
top 4: {'accuracy': 0.9375}


Dataset({
    features: ['text', 'class_code'],
    num_rows: 16
})

In [17]:
df_train

,text,class_code
38,Cardinals Outslug Astros to Take Opening Game ...,2
13,Amvescap reorganizes after settling The chairm...,0
50,Approval Expected for Big Cellphone Deal Feder...,1
10,Pope Prays for Beslan School Dead at Italy Mas...,3
11,Possible playoff preview A #39;s take a hit in...,2
31,Rapids Clinch MLS Playoff Berth (AP) AP - Dwya...,2
68,Pacers triumph in Harrington #39;s return Harr...,2
75,UK minister visits Arafat grave UK Foreign Sec...,3
58,Peru Rebel Chief Scores Publicity Coup in Cour...,3
24,US Airways wants court to cut pay Troubled car...,0


In [18]:
### build trainer for contrastive learning
model.buildTrainer(train_dataset)
### train
model.contrastive_train()
gc.collect()

model.predict_proba(test_dataset['text'])

Applying column mapping to training dataset
***** Running training *****
  Num examples = 64
  Num epochs = 1
  Total optimization steps = 16
  Total train batch size = 4


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/16 [00:00<?, ?it/s]

Predicting probabilities of 92 documents


array([[0.1613198 , 0.63082488, 0.09959159, 0.10826372],
       [0.17862996, 0.35241319, 0.24500873, 0.22394813],
       [0.26222869, 0.39871494, 0.10941236, 0.22964401],
       [0.14502072, 0.18460921, 0.1410485 , 0.52932157],
       [0.16789531, 0.25031984, 0.19643049, 0.38535436],
       [0.17250024, 0.29742212, 0.28099278, 0.24908486],
       [0.21184902, 0.29497208, 0.15129715, 0.34188175],
       [0.17010913, 0.61777496, 0.10147623, 0.11063968],
       [0.10321494, 0.14128653, 0.62483142, 0.13066711],
       [0.29873229, 0.40385709, 0.12960178, 0.16780884],
       [0.10195986, 0.13416726, 0.13113685, 0.63273604],
       [0.11822164, 0.14686209, 0.60519831, 0.12971797],
       [0.12317564, 0.1935894 , 0.20333218, 0.47990278],
       [0.44645636, 0.27635052, 0.12318874, 0.15400438],
       [0.19483485, 0.50645699, 0.15185638, 0.14685178],
       [0.17623696, 0.23708444, 0.35151165, 0.23516696],
       [0.27689675, 0.4465292 , 0.13619857, 0.14037548],
       [0.29526445, 0.44376727,

In [19]:
setfit_all_metrics = evaluation_metrics.get_metrics(model.y_pred, test_dataset["class_code"])
print(setfit_all_metrics)
print(classification_report(model.y_pred, test_dataset['class_code'],target_names=classes_list))
setfit_exec_time  = evaluation_metrics.saveResults(model.config,setfit_all_metrics,None)
print(model.config)

{'weighted': [{'accuracy': 0.6304347826086957}, {'precision': 0.808209906035993}, {'recall': 0.6304347826086957}, {'f1': 0.5865247585876633}], 'macro': [{'accuracy': 0.6304347826086957}, {'precision': 0.8081501831501832}, {'recall': 0.6467948717948717}, {'f1': 0.597690980669704}]}
                        precision    recall  f1-score   support

              business       0.08      1.00      0.15         2
science and technology       1.00      0.42      0.59        52
                sports       0.85      1.00      0.92        17
                 world       0.65      0.81      0.72        21

              accuracy                           0.63        92
             macro avg       0.65      0.81      0.60        92
          weighted avg       0.87      0.63      0.67        92

metrics_setfit_2023_04_17__20_34_53.csv
config_setfit_2023_04_17__20_34_53.csv
{'similarity_model': 'sentence-transformers/all-roberta-large-v1', 'setfit_model': 'sentence-transformers/all-roberta-large-